## [NYCDSA Capstone Project] 
# Women's Softball League Power Ranking Estimate

<br>
Koeun Lim (koeunlim@alum.mit.edu)<br>
Kevin Haghi (kevin.haghi@gmail.com)<br>


# Step 8. Deploy

---
## Project Description



### Project Outline
- Step 1. Web scraping
- Step 2. Clean data
- Step 3. EDA
- Step 4. Imputation & PCA
- Step 5-1. Modeling (1) RPI prediction based on stats - XGboost regression
- Step 5-2. Modeling (1) RPI prediction based on stats - Linear regression
- Step 6-1. Modeling (2) WS 64 Seed prediction based on stats - XGboost classification
- Step 6-2. Modeling (2) WS 64 Seed prediction based on stats - Logistic regression

In [9]:
import pandas as pd

import sqlite3
from sqlite3 import Error

from sqlalchemy import create_engine

In [3]:
# Connect to sqlite


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(r"/Users/koeunlim/Dropbox (Personal)/NYCDSA/Project_Capstone/Capstone_Softball/sqlite/db/pythonsqlite.db")

2.6.0


In [4]:
def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

In [12]:
df_RPI = pd.read_csv('LM_RPI/RPI_prediction_logit_1000.csv')
df_RPI

,Oklahoma,UCLA,Washington,Arizona,Florida State,Alabama,Minnesota,Texas,Oklahoma State,Florida,...,Southern Utah,Coppin State,Savannah State,State Bonaventure,Grambling,Ark.-Pine Bluff,UMES,Southern U.,Saint Peter's,South Carolina State
0,1.0,2.0,5.0,3.0,4.0,7.0,14.0,16.0,22.0,6.0,...,266.0,290.0,282.0,280.0,285.0,296.0,295.0,288.0,293.0,297.0
1,1.0,2.0,3.0,5.0,4.0,8.0,14.0,13.0,21.0,6.0,...,267.0,290.0,283.0,275.0,285.0,296.0,295.0,288.0,294.0,297.0
2,1.0,2.0,3.0,4.0,5.0,7.0,12.0,13.0,22.0,6.0,...,273.0,292.0,284.0,276.0,283.0,295.0,296.0,286.0,293.0,297.0
3,1.0,2.0,3.0,6.0,4.0,8.0,15.0,12.0,21.0,5.0,...,264.0,292.0,285.0,275.0,282.0,296.0,295.0,287.0,293.0,297.0
4,1.0,2.0,7.0,4.0,3.0,6.0,14.0,17.0,22.0,5.0,...,273.0,292.0,284.0,281.0,283.0,296.0,295.0,288.0,291.0,297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,2.0,4.0,3.0,5.0,8.0,13.0,16.0,21.0,7.0,...,271.0,292.0,284.0,280.0,282.0,296.0,295.0,287.0,293.0,297.0
996,1.0,2.0,3.0,4.0,5.0,7.0,11.0,18.0,26.0,6.0,...,270.0,292.0,284.0,278.0,282.0,296.0,295.0,287.0,293.0,297.0
997,1.0,2.0,3.0,4.0,5.0,9.0,13.0,15.0,26.0,6.0,...,265.0,291.0,286.0,279.0,282.0,296.0,295.0,287.0,293.0,297.0
998,1.0,2.0,4.0,5.0,3.0,9.0,13.0,16.0,27.0,7.0,...,273.0,290.0,282.0,276.0,285.0,296.0,295.0,289.0,292.0,297.0


In [25]:
engine = create_engine('sqlite:///sqlite/db/RPI_2019.db', echo=True)
sqlite_connection = engine.connect()

2021-03-19 00:18:36,095 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-19 00:18:36,097 INFO sqlalchemy.engine.base.Engine ()
2021-03-19 00:18:36,101 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-19 00:18:36,102 INFO sqlalchemy.engine.base.Engine ()


In [26]:
sqlite_table = "RPI2019"
df_RPI.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-19 00:18:38,040 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("RPI2019")
2021-03-19 00:18:38,041 INFO sqlalchemy.engine.base.Engine ()
2021-03-19 00:18:38,042 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("RPI2019")
2021-03-19 00:18:38,043 INFO sqlalchemy.engine.base.Engine ()
2021-03-19 00:18:38,090 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "RPI2019" (
	"index" BIGINT, 
	"Oklahoma" FLOAT, 
	"UCLA" FLOAT, 
	"Washington" FLOAT, 
	"Arizona" FLOAT, 
	"Florida State" FLOAT, 
	"Alabama" FLOAT, 
	"Minnesota" FLOAT, 
	"Texas" FLOAT, 
	"Oklahoma State" FLOAT, 
	"Florida" FLOAT, 
	"James Madison" FLOAT, 
	"Northwestern" FLOAT, 
	"LSU" FLOAT, 
	"Michigan" FLOAT, 
	"Ole Miss" FLOAT, 
	"Kentucky" FLOAT, 
	"Tennessee" FLOAT, 
	"ULL" FLOAT, 
	"Texas Tech" FLOAT, 
	"Auburn" FLOAT, 
	"North Carolina" FLOAT, 
	"Drake" FLOAT, 
	"South Carolina" FLOAT, 
	"Arizona State" FLOAT, 
	"Georgia" FLOAT, 
	"Missouri" FLOAT, 
	"Virginia Tech" FLOAT, 
	"Mississippi State" FLOA

2021-03-19 00:18:38,548 INFO sqlalchemy.engine.base.Engine ((0, 1.0, 2.0, 5.0, 3.0, 4.0, 7.0, 14.0, 16.0, 22.0, 6.0, 15.0, 25.0, 11.0, 19.0, 18.0, 17.0, 12.0, 9.0, 21.0, 13.0, 33.0, 34.0, 26.0, 38.0, 10.0, 24.0, 8.0, 29.0, 20.0, 71.0, 48.0, 54.0, 28.0, 40.0, 41.0, 37.0, 27.0, 31.0, 23.0, 59.0, 64.0, 66.0, 55.0, 52.0, 36.0, 47.0, 46.0, 53.0, 73.0, 42.0, 75.0, 44.0, 65.0, 63.0, 43.0, 84.0, 39.0, 105.0, 50.0, 86.0, 57.0, 87.0, 51.0, 72.0, 62.0, 35.0, 93.0, 32.0, 45.0, 112.0, 134.0, 111.0, 108.0, 92.0, 127.0, 58.0, 99.0, 81.0, 77.0, 56.0, 30.0, 70.0, 67.0, 82.0, 121.0, 94.0, 110.0, 90.0, 69.0, 85.0, 115.0, 74.0, 49.0, 68.0, 95.0, 80.0, 109.0, 60.0, 175.0, 125.0, 97.0, 104.0, 78.0, 106.0, 89.0, 100.0, 107.0, 174.0, 139.0, 61.0, 114.0, 128.0, 141.0, 182.0, 101.0, 164.0, 186.0, 117.0, 140.0, 153.0, 116.0, 98.0, 156.0, 102.0, 119.0, 142.0, 148.0, 137.0, 149.0, 79.0, 132.0, 154.0, 103.0, 120.0, 165.0, 122.0, 188.0, 133.0, 136.0, 170.0, 155.0, 118.0, 113.0, 223.0, 157.0, 138.0, 159.0, 129.0, 131

2021-03-19 00:18:38,619 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-19 00:18:38,626 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-03-19 00:18:38,627 INFO sqlalchemy.engine.base.Engine ()


In [27]:
sqlite_connection.close()